In [1]:
import torch
from transformers import (
    AutoProcessor,
    AutoModelForSpeechSeq2Seq,
    pipeline,
)
from peft import PeftModel
import librosa
import yt_dlp
import json
import re
import os
from jiwer import wer

/home/shaider/miniconda3/envs/whisper-lora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/shaider/miniconda3/envs/whisper-lora/lib/python3.10/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
# Update this to wherever your LoRA adapter folder is
LORA_DIR = "/home/shaider/whisper-UrduASR/fyp-summer/finetuning_experimentation/finetuning-33/lora_adapter" 
COOKIE_FILE = "/home/shaider/Downloads/cookies.txt"   # only if you uploaded cookies

In [3]:
BASE_MODEL = "openai/whisper-large-v3-turbo"
compute_dtype = torch.float16  # Or torch.bfloat16 for stability on some GPUs

print("Loading base Whisper model without quantization...")
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    BASE_MODEL,
    device_map="auto",  # Automatically places on GPU
    torch_dtype=compute_dtype,
    low_cpu_mem_usage=True,
    attn_implementation="sdpa",  # For speed
)

print("Loading LoRA...")
model = PeftModel.from_pretrained(model, LORA_DIR)

print("Merging LoRA...")
model = model.merge_and_unload()
model.eval()

processor = AutoProcessor.from_pretrained(BASE_MODEL)

print("✅ Model loaded on GPU without bitsandbytes!")

Loading base Whisper model without quantization...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading LoRA...
Merging LoRA...
✅ Model loaded on GPU without bitsandbytes!


In [4]:
import json
from pathlib import Path

cfg_path = Path(LORA_DIR) / "best_generation_config.json"

valid_keys = {
    "num_beams","length_penalty","repetition_penalty","do_sample",
    "max_new_tokens","temperature","top_p","top_k","no_repeat_ngram_size",
}

if cfg_path.exists():
    with cfg_path.open("r", encoding="utf-8") as f:
        raw_params = json.load(f)
    decoding_params = {k: v for k, v in raw_params.items() if k in valid_keys}
    print("Using decoding params from file:", decoding_params)
else:
    decoding_params = {}
    print(f"⚠️ best_generation_config.json not found at: {cfg_path}")
    print("Proceeding with default decoding params (empty dict).")

Using decoding params from file: {'num_beams': 3, 'length_penalty': 1.0, 'do_sample': False, 'max_new_tokens': 225}


In [5]:
def fix_youtube_url(url):
    # Convert shorts → watch?v=
    if "shorts/" in url:
        return url.replace("shorts/", "watch?v=")
    return url

def download_audio(url, out="audio"):
    url = fix_youtube_url(url)

    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": out,
        "quiet": True,
        "cookiefile": COOKIE_FILE,
        "postprocessors": [{"key": "FFmpegExtractAudio", "preferredcodec": "wav"}],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    if os.path.exists(out + ".wav"):
        return out + ".wav"

    raise FileNotFoundError("Audio download failed.")


In [6]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,

    # THIS IS THE ONLY CONFIG THAT WORKS RELIABLY FOR 500+ SECONDS
    chunk_length_s=None,           # ← 30-second chunks with internal overlap
    #batch_size=8,                # ← 8–16 works on 24GB GPU
    return_timestamps="word",    # ← Gives you word-level timestamps + generate_with_fallback

    generate_kwargs={
        "language": "ur",
        "task": "transcribe",
        # DO NOT SET max_new_tokens here when using chunking!
        # The pipeline automatically handles it per chunk
    }
)

print("FINAL CONFIG – works on 227s AND 500+ second Urdu files without truncation")

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


FINAL CONFIG – works on 227s AND 500+ second Urdu files without truncation


In [7]:
import numpy as np  # For concatenation/slicing (if not imported)

def transcribe_long_audio(path):
    print(f"Loading {path} ...")
    audio, sr = librosa.load(path, sr=16000, mono=True)
    duration = len(audio) / sr
    print(f"Audio duration: {duration:.1f}s")
    
    # No chunking (since that's giving best results)
    chunk_length_s = None
    stride_length_s = None
    
    # Transcribe main audio
    result = pipe(audio)
    full_text = result["text"].strip()
    
    # Check if end was truncated (last timestamp < duration - tolerance)
    tolerance = 5.0  # Allow up to 5s drop (adjust if needed)
    if "chunks" in result and result["chunks"]:
        last_timestamp = result["chunks"][-1]["timestamp"][1]
        if last_timestamp < duration - tolerance:
            print(f"Detected truncation at {last_timestamp:.1f}s – transcribing last {duration - last_timestamp:.1f}s separately")
            
            # Slice last partial segment (starting from last_timestamp - overlap)
            overlap = 2.0  # 5s overlap to stitch smoothly
            start_sample = int(max(0, last_timestamp - overlap) * sr)
            partial_audio = audio[start_sample:]
            
            # Transcribe partial end
            partial_result = pipe(partial_audio)
            partial_text = partial_result["text"].strip()
            
            # Stitch: Append partial text (remove overlap repetition if any)
            full_text += " " + partial_text  # Simple concat; can refine if overlap duplicates
            
            print(f"Appended partial end – new transcribed up to {duration:.1f}s")
    
    return full_text.strip()

In [8]:
def evaluate(urls, gold_texts):
    preds, wers_list = [], []

    for url, gold in zip(urls, gold_texts):
        print(f"\n⬇️ Downloading audio: {url}")
        audio_path = download_audio(url)

        print("🎧 Transcribing…")
        pred = transcribe_long_audio(audio_path)
        preds.append(pred)

        w = wer(gold.lower(), pred.lower())
        wers_list.append(w)

        print("\n===== RESULT =====")
        print("GOLD:", gold)
        print("PRED:", pred)
        print("WER:", w)

    return preds, wers_list





In [9]:
# # ============================================
# # 8) YOUR LINKS + GOLD TRANSCRIPTS
# # ============================================
# youtube_links = [
#     #"https://youtube.com/shorts/XRk8kpNFWBQ?si=L1RI4xHIQv4T6hu3",
#     #"https://youtube.com/shorts/5CLY0cdvQPw?si=3BHgNKAs448aaDyV",
#     #"https://youtube.com/shorts/9cxQHuaf0VE?si=JnuhSkHVwQJekNI9",
#     #"https://youtube.com/shorts/9sKr42cXPrk?si=or6wJ5zHY39lmXpE",
#     "https://youtube.com/shorts/P5RGCMgXBLo?si=7HUobGTy-BvXVAfh"
# ]

# gold_transcripts = [
#     #"اضافی بجلی کے استعمال کے لیے صارفین کو راغب کرنے کا مشن بجلی کی قیمتوں میں بڑی کمی کا امکان ہے زرعی اور صنعتی صارفین کے لیے طویل المدتی ریلیف پیکج کی تیاری پاور ڈویزن ریلیف پیکج منظوری کے لیے وزیراعظم کو پیش کرے گا وزیراعظم آئندہ چند روز میں ریلیف پیکج کا اعلان بھی کریں گے ظہیر علی ہمارے ساتھ موجود ہیں ظہیر بتائے گا کتنا ریلیف مل سکتا ہے صارفین کو کیا ڈیٹیلز ہیں آپ کے پاس دیکھیں اس وقت جو سردیوں میں بجلی ہے اس کی کھپت پہلے سے کم ہو جاتی ہے اس کو بڑھانے کے لیے پچھلے سال بھی ایک ریلیف پیکج دیا گیا تھا جسے ونٹر ریلیف پیکج کا نام دیا گیا تھا جس کی وجہ سے بجلی کی جو اضافی بجلی ہے اس کی کھپت میں اضافہ ہوا تھا اس وقت بھی ایک ریلیف پیکج تیار کر لیا گیا ہے اور پرائم منسٹر آف پاکستان اس کی باقاعدہ طور پر منظوری دیں گے جس میں زرعی صنعتی صارفین کے لیے یہ پیکج لایا جائے گا چونکہ اس وقت جو نرخ ہے جو زرعی صارفین کے لیے 38 روپے اور صنعتی صارفین کے لیے 34 روپے پر یونٹ کو ادا کر رہے ہیں تو کوشش کی جارہی ہے کہ اس کو کمپیٹیٹیو کیا جائے اور کم سے کم کیا جائے تاکہ نیشن گرڈ پر صارفین کا جو بہت شکریہ ظہیر علی خان",
#     #"اہم خبر دیں پی ایس ایل فرنچائز ملتان سلطانز کے اونر علی ترین نے پی سی بی کا لیگل نوٹس پھاڑ دیا ویڈیو بیان میں علی ترین نے کہا لیگل نوٹس میں تنقیدی بیانات واپس لینے اور معافی مانگنے کا مطالبہ کیا گیا ہے فرنچائز معاہدہ ختم کرنے اور بلیک لسٹ کرنے کی دھمکی دی گئی لیکن میں یہ دھمکیوں سے خاموش نہیں ہوں گا پی ایس ایل فرنچائز ملتان سلطانز کے اونر علی ترین نے پی سی بی کا لیگل نوٹس پھاڑ دیا ویڈیو بیان میں علی ترین نے کہا لیگل نوٹس میں تنقیدی بیانات واپس لینے اور معافی مانگنے کا مطالبہ کیا گیا فرنچائز معاہدہ ختم کرنے اور بلیک لسٹ کرنے کی دھمکی دی گئی لیکن میں یہ دھمکیوں سے خاموش نہیں ہوں گا پی ایس ایل فرنچائز ملتان سلطانز کے اونر علی ترین نے پی سی بی کا لیگل نوٹس پھار دیا ویڈیو بیان میں علی ترین نے کہا لیگل نوٹس میں تنقیدی بیانات واپس لینے اور معافی مانگنے کا مطالبہ کیا گیا فرنچائز معاہدہ ختم کرنے اور بلیک لسٹ کرنے کی دھمکی دی گئی لیکن میں یہ دھمکیوں سے خاموش نہیں ہوں گا پی ایس ایل فرنچائز ملتان سلطانز کے اونر علی ترین نے پی سی بی کا لیگل نوٹس پھار دیا ویڈیو بیان میں علی ترین نے کہا لیگل نوٹس میں تنقیدی بیانات واپس لینے اور معافی مانگنے کا مطالبہ کیا گیا علی ترین کا مزید کیا کہنا تھا آئیے آپ کو دکھاتے ہیں پی ایس ایل مینجمنٹ نے مجھے ایک لیگل نوٹس بھیجا جس میں وہ کہہ رہے ہیں کہ اگر میں نے اپنی تمام باتیں جو میں نے پی ایس ایل مینجمنٹ کے خلاف کی ہیں وہ واپس نہ لیں اور پی ایس ایل مینجمنٹ کو ایک پبلک اپالوجی نہ کی تو وہ ہمارا ملتان سلطان فرانچائز ایگریمنٹ کینسل کر دیں گے اور مجھے ہمیشہ کے لیے بلیک لیسٹ کر دیں گے پی ایس ایل سے تاکہ میں کبھی بھی کوئی ٹیم نہ لے سکو دیکھیں یہی تو ایشو ہے آپ لوگ چاہتے نہیں ہیں کہ آپ لوگ اپنے اسٹیک ہولڈرز کے ساتھ کام کریں آپ بس چاہتے ہیں کہ ہم سب چپ کر کے بیٹھیں اور آپ کی لائن ٹو کریں آپ لوگ اتنا یوز ٹو ہیں کہ آپ کے اردگرد یس من ہیں اور منینز ہیں کہ آپ تھوڑی سی بھی کریٹیسزم برداشت نہیں کر سکتے میں نے آج تک ایک کال ایک میسیج ایک ایمیل ایک انویٹیشن ریسیو نہیں کی کہ کیا آپ کے ایشوز ہیں آئیں دفتر میں آئیں اکٹھے مل کر ہم ان کو ایڈریس کرتے ہیں اور دیکھتے ہیں کہ ہم کیا بہتری لاسکتے ہیں اس لیگ میں سیدھا لیگل نوٹس پھر آپ لوگ تھوڑے سے بھی کمپیٹنٹ ہوتے تو آپ کو پتا ہوتا ہے کہ ایسے چیزیں مینیج نہی ہوتی ہیں اور دیکھیں اگر آپ کو لگتا ہے کہ اس طرح کے تھریٹ سے میں چپ ھو جاؤں گا یہ آپ کے غلط فہمی ہے کیونکہ میں اس لیگ کو لو کرتا ہوں میں اپولوجائز کرتا ہوں کہ میں نے اتنی یہ غلط چیزیں آپ لوگوں سے ریکویسٹ کی اور غلط چیزوں پر میں نے تنقید کی سو میں اپولوجی کرتا ھوں آئ ھوپ یو لائک مائ اپولوجی ویڈیو ود لو علی", 
#     #" بریکنگ نیوز ہے کراچی کے علاقے ناظم آباد میں گاڑی کی ٹکر سے ایک شخص جاں بحق ہو گیا ہے اس پر مزید کیا تفصیلات ہیں جانیں گے کراچی میں جیو نیوز کے نمائندے قمر علی مستوئی سے قمر بتائے گا حادثہ کیسے پیش آیا ہے اور جاں بحق شخص کی شناخت ہو سکی ہے کیا؟ جی اس حوالے سے ریسکیو کا کہنا ہے کہ نامعلوم گاڑی کی ٹکر سےجو ہے وہ یہ شخص جاں بحق ہوا ہے جس کی فوری طور پر شناخت نہیں ہو سکی ہے جاں بحق ہو نے والے شخص کی لاش کو عباسی شہید ہسپتال منتقل کیا گیا ہے اس کے کپڑوں سے کوئی شناختی دستاویزات جو ہے وہ نہیں ملی پولیس نے جو ہے وہ متوفی کو ٹکر مارنے والی گاڑی کی تلاش شروع کر دی ہے اور جائے حادثہ  سےجو ہےوہ پولیس کی اطراف میں موجود شہریوں سےجو ہے وہ مرنے والے شخص کے متعلق معلومات حاصل کرنے کی کوشش کی کر رہی ہے تاکہ اس کی شناخت ہو سکے دوسری جانب بھینس کالونی نمبر دس پر بھی جو ہے وہ ٹریفک حادثہ پیش آیا ہے پولیس کے مطابق جو ہے وہ تیز رفتار ٹرالر ڈرائیور کے قابو سے باہر ہو کر الٹ گیا جس کے نتیجے میں ایک شخص زخمی ہواہے زخمی شخص کو جناح ہسپتال منتقل کر دیا گیا ہے علاقہ پولیس اور ٹریفک پیش پہنچ کر ہیوی مشینیز کی مدد سے جو ہے وہ ٹرک کو سڑک سے جو ہے سائیڈ پر کر دیا ہے پولیس کاکہنا کہ واقعہ کی تحقیقات کر رہے ہیں زخمی ہونے والا  شخص سے بھی اس حادثے سے متعلق معلومات لی جا رہی ہیں بہت شکریہ قمر آپ نے اپ ڈیٹ کیا",
#     #" شدت پسندی اور اسلام کی من گھڑت تشریحات کا خاتمہ ضروری ہے فیلڈ مارشل عاصم منیر کا دورہ مصر میں بڑا بیان مصر کی دفاعی قیادت سے اہم ملاقاتیں برادر اسلامی ملکوں کے تعاون کو علاقائی امن و استحکام اور مفاد عامہ کا حامل قرار دیا سپاہ سالار جامعتہ الاظہر کے شیخ اعظم احمد الطیب سے بھی ملے پاک مصر فوجی تعاون اور دفاعی اشتراک مزید مضبوط بنانے کا مشن فیلڈ مارشل سید عاصم منیر کا سرکاری دورہ مصر میزبان وزیر دفاع و دفاعی پیداواج جنرل عبدالمجید سقر سے ملاقات مصری مسلح افواج کے چیف آف اسٹاف لیفٹننٹ جنرل احمد خلیفہ فتحی سے بھی ملے مصری وزارت دفاع پہنچنے پر فیلڈ مارشل کا پرتپاک استقبال سلامی دی گئی مصری دفاعی قیادت سے بیٹھک میں باہمی دلچسپی کے امور علاقائ سلامتی کی صورتحال اور دفاعی تعاون پر تفصیلی تبادلہ خیال کیا گیا جنرل عاصم منیر نے کہا مصر برادر اسلامی ملک اور دونوں ممالک کا تعاون عوامی مفاد میں ہے دو طرف تعاون خطے میں امن و استحکام کے فروغ میں بھی مددگار قرار فیلڈ مارشل نے یادگار اور سابق مصری صدر انور سادات کی قبر پر پھول بھی رکھے فیلڈ مارشل عاصم منیر نے جامعتہ الاظہر کے شیخ اعظم احمد الطیب سے بھی ملاقات کی شیخ جامعتہ الاظہر کا مسلم امہ کو درپیش چیلنجز پر اظہار خیال فیلڈ مارشل عاصم منیر نے شدت پسندی اور اسلام کی من گھڑت تشریحات کے خاتمے کی ضرورت پر زور دیا",
#     " عمران خان کی روز اتنی میٹنگز ہوتی ہیں آپ ان کو جیل ملنے گئے ہیں کبھی؟ یہ بڑا سیریس کوئسچن ہے عمران خان صاحب نے دو تین دفعہ جو لسٹ بنتی ہے اس میں میرا نام ڈلوایا وہ مجھے ملنا چاہتے تھے لیکن جو سرکاری حکام ہیں جو میٹنگ کے مالک ہیں انہوں نے میری میٹنگ نہیں ہونے دی لیکن آپ چاہتے تھے ان سے ملنا بھی نہیں وہ چاہتے تھے مجھ سے ملنا اور اگر وہ چاہتے تھے تو میں ایک جنرلسٹ ہوں اور میں ان سے ملنے کے لیے تیار تھا مثلا میں آپ کو ایک مثال دیتا ہوں نا کہ ایک دن ان کے ایک وکیل نے مجھے کال کی اور اس نے کہا جی کہ آپ نے کوئی سنا ھے کہ پٹیشن فائل کی ہے اسلام آباد ہائی کورٹ میں ارشد شریف کے مرڈر کیس میں کوئی آپ چاہتے ہیں کہ جوڈیشل کمیشن بن جائے میں نے کہا ہاں تو اس نے کہا عمران خان صاحب کو پتا چلا ہے وہ آپ کے ساتھ کچھ انفارمیشن شیئر کرنا چاہتے ہیں کیونکہ وہ ان کو کچھ پتا ہے تو میں نے کہا ٹھیک ہے تو میں نے کہا پھر میں ان کو کیسے ملوں گا تو انہوں نے کہا جی کہ مجھے انہوں نے انسٹرکشن دیا ہے تو میں آپ کا نام لسٹ میں ڈال رہا ہوں عمران خان صاحب کی ہدایت انہوں نے میرا نام ڈالا مجھے کال بھی آئی میں اڈیالہ پہنچ بھی گیا مجھے ملنے نہیں دیا کسی نے"
# ]



In [10]:
from pathlib import Path
MODEL_COLUMN = "Whisper Turbo FT"   # <-- set ONE of these exactly as in sheet:
# "Whisper Large"
# "Whisper Large FT"
# "Whisper turbo"
# "Whisper turbo FT"

TRANSCRIPTS_ROOT = Path("/home/shaider/whisper-UrduASR/fyp-summer/youtube_videos_dataset")

In [11]:
import os
print(os.getcwd())


/home/shaider/whisper-UrduASR


In [12]:
from pathlib import Path
import pandas as pd

# ====== CONFIG ======
CSV_PATH = Path("fyp-summer/youtube_videos_dataset/youtube_videos.csv")   # your downloaded sheet
GOLD_DIR = Path("fyp-summer/youtube_videos_dataset/gold")                        # folder where gold .txt files live
NAME_COL = "Name"
URL_COL  = "youtube_url"
GOLD_FILE_COL = "gold_path"                   # put this column in the sheet
# ====================

def load_manifest(csv_path: Path):
    df = pd.read_csv(csv_path)

    # basic validation
    for col in [NAME_COL, URL_COL, GOLD_FILE_COL]:
        if col not in df.columns:
            raise ValueError(f"Missing column '{col}' in {csv_path}. Columns: {list(df.columns)}")

    # clean rows (drop empty url/gold_file)
    df = df.dropna(subset=[URL_COL, GOLD_FILE_COL]).copy()
    df[URL_COL] = df[URL_COL].astype(str).str.strip()
    df[GOLD_FILE_COL] = df[GOLD_FILE_COL].astype(str).str.strip()
    df[NAME_COL] = df[NAME_COL].astype(str).str.strip()

    youtube_links = df[URL_COL].tolist()
    gold_paths = [GOLD_DIR / fn for fn in df[GOLD_FILE_COL].tolist()]

    gold_transcripts = []
    missing = []
    for p in gold_paths:
        if not p.exists():
            missing.append(str(p))
            gold_transcripts.append("")  # placeholder, but we will error below
        else:
            gold_transcripts.append(p.read_text(encoding="utf-8", errors="replace"))

    if missing:
        raise FileNotFoundError(
            "Some gold files are missing in your GOLD_DIR.\n"
            + "\n".join(missing)
        )

    names = df[NAME_COL].tolist()
    return names, youtube_links, gold_transcripts, df

names, youtube_links, gold_transcripts, manifest_df = load_manifest(CSV_PATH)

print(f"Loaded {len(youtube_links)} samples from {CSV_PATH}")
print("First 3:")
for i in range(min(3, len(names))):
    print(f"- {names[i]} | {youtube_links[i]} | {manifest_df.loc[i, GOLD_FILE_COL]}")

Loaded 30 samples from fyp-summer/youtube_videos_dataset/youtube_videos.csv
First 3:
- 11Death | https://www.youtube.com/shorts/oaUpDIVXtlk | 11Deaths Gold.txt
- 3Alleged | https://www.youtube.com/shorts/11bwoKOjESY | 3Alleged Gold.txt
- AfghanCricket | https://www.youtube.com/shorts/HhcIFYNjYw0 | AfghanCricketGPT.txt


In [13]:
# ============================================
# 9) RUN EVALUATION
# ============================================
preds, wers_list = evaluate(youtube_links, gold_transcripts)

print("\====== SUMMARY ======")
for i, (p, w) in enumerate(zip(preds, wers_list)):
    print(f"[{i}] WER = {w:.4f}")


⬇️ Downloading audio: https://www.youtube.com/shorts/oaUpDIVXtlk


🎧 Transcribing…                                           
Loading audio.wav ...
Audio duration: 107.1s

===== RESULT =====
GOLD: افسوسناک خبر شیئر کریں گے خیرپور ٹنڈو مستی کے قریب قومی شاہراہ پر مسافر کوچ اور ٹرالر میں ٹکر حادثے میں گیارہ افراد جاں بحق دس زخمی ہو گئے جاں بحق افراد میں کوچ کا ڈرائیور اور کنڈکٹر بھی شامل ہیں اس پر مزید کیا ڈیٹیل ہیں یہ جانئینگے جیو نیوز کے نمائندے یاسر فاروقی سے یاسر بتائیے گا یہ حادثہ کیسے پیش آیا جی بالکل خیرپور میں قومی شاہراہ ٹنڈو مستی کے قریب یہ حادثہ پیش آیا ہے جہاں پے تیز رفتار مسافر کوچ اوور ٹیکنگ کرتے ہوئےجو ہے ٹیئر گارڈر مَے سے بھرے ٹریلر میں جا گھسی ہے جس کے باعث گیارہ افراد جاں بحق ہوئے ہیں دس سے زائد افراد زخمی ہیں زخمیوں اورلاش لاشوں کو سول ہسپتال خیرپور اور گمز ہسپتال گمبٹ منتقل کیا جا رہا ہے حادثے میں مسافر کوچ کا ڈرائیور اور کنڈکٹر بھی جاں بحق ہوا ہے یہ بتایا جا رہا ہے کہ اوور ٹیکنگ کے دوران یہ حادثہ پیش آیا ہے جائے حادثہ پے اس وقت بھی ریسکیو آپریشن ایڈی اور دیگر تنظیموں کی جانب سے خیرپور پولیس کی جانب سے جاری ہے جب کے یہ مسافر کوچ پنجا

🎧 Transcribing…                                           
Loading audio.wav ...
Audio duration: 82.2s

===== RESULT =====
GOLD: آغاز کریں گے کراچی کے علاقے کورنگی چکرا گوٹھ میں رینجرز اور سی ٹی ڈی نے کارروائی کے دوران پتل ہندوستان سے تعلق رکھنے والے تین سہولت کاروں کو گرفتار کر لیا ہے آپ کو اپ ڈیٹ کرتے چلیں کہ کراچی کے علاقے کورنگی چکرا گوٹھ میں رینجرز اور سی ٹی ڈی نے کارروائی کی ہے جس کے دوران پتل ہندوستان سے تعلق رکھنے والے تین سہولت کاروں کو گرفتار کر لیا گیا ہے سی ٹی ڈی کے جانب سے کراچی کے علاقے کورنگی چکرا گوٹھ میں رینجرز اور سی ٹی ڈی کے جانب سے کاروائی کے دوران پتل ہندوستان سے تعلق رکھنے والے تین سہولت کاروں کو گرفتار کر لیا گیا ہے یہ کاروائی کی گئی ہے کورنگی چکرا گوٹھ میں جہاں پے رینجرز اور سی ڈی ٹی کی مشترکہ کاروائی تھی ملزمان سے  لیپ ٹاپ جب کہ پانچ کلو بارودی مواد بھی برآمد کر لیا گیا ہے آپ کو یہاں  اپڈیٹ کرتے چلیں کراچی کے علاقے میں کارروائی کی گئی ہے اور قاضی حسن سے مزید بات کرتے ہیں قاضی حسن بتائیے گا اس کاروائی کے حوالے سے کیا مزید  تفصیلات ہیں جی یہ کاروائی رینجرز اور سی

🎧 Transcribing…                                           
Loading audio.wav ...
Audio duration: 62.3s

===== RESULT =====
GOLD: آئی سی سی ٹی ٹوئنٹی ورلڈ کپ میں سری لنکا نے آسٹریلیا کو آٹھ وکٹوں سے ہرا دیا انگلینڈ نے اٹلی کے خلاف چوبیس رنز سے فتح حاصل کی جبکہ افغانستان نے یو اے ای کو پانچ وکٹوں سے شکست دی ایونٹ کے میچز جیو سوپر اور ڈیجیٹل ایپ مائیکو سے براہ راست نشر کیے جا رہے ہیں کولمبو میں کھیلے گئے میچ میں آسٹریلوی اوپنرز ٹریوس ہیڈ اور مچل مارش نے پہلی وکٹ پر ٹیم کو ایک سو چار رنز کا آغاز فراہم کیا ٹریوس ہیڈ چھپن اور مچل مارش چون رنز بنا کر آؤٹ ہوئے جس کے بعد آسٹریلیا کی وکٹیں وقفے وقفے سے گرتی گئیں اور پوری ٹیم ایک سو اکیاسی رنز بنا کر آؤٹ ہو گئی  ہیمان خالد   نے تین اور دشمان تا جمیلاہ نے دو کھلاڑیوں کو آؤٹ کیا جواب میں اوپنر پاتھم نسانکا کی سو ناٹ آؤٹ اننگز کی بدولت سری لنکا نے ہدف انیسویں اوور میں حاصل کر لیا کوشل مینڈس نے اکیاون رنز بنائے اس سے قبل کھیلے گئے میچ میں انگلینڈ نے اٹلی کو چوبیس رنز سے شکست دے کر سپر ایٹ مرحلے کے لیے کوالیفائی کر لیا انگلینڈ کے دو سو دو رنز کے جواب 

🎧 Transcribing…                                           
Loading audio.wav ...
Audio duration: 118.9s

===== RESULT =====
GOLD: سولجر بازار میں گل رعنا کالونی کی جو ایک عمارت تھی جو تین منزلہ عمارت تھی اس میں ریسکیو کا آپریشن مکمل کر لیا گیا ہے جو عزیز و اقارب تھے انہوں نے جو لسٹ پروائد کی تھی انتظامیہ کو اسکے مطابق تمام باڈیز جو ہیں وہ نکال لی گئی ہیں اور مجموعی طور پر پندرہ لاشیں جو ہیں وہ سول اسپتال منتقل کی گئی ہیں چودہ افراد  جو ہیں وہ زخمی حالت میں یہاں سے نکال کر سول اسپتال میں طبی امداد دی جا رہی ہے اس وقت جب ڈی سی ایسٹ سے ہماری بات ہوئی تو انکا یہ کہنا تھا کے ریسکیو آپریشن جو ہے وہ ختم ہو چکا ہے تاہم عمارت جو کے اسٹرکچر اس وقت موجود ہے اسے گرانے کا کام جاری ہے اور اس عمارت کو گراؤند کیا جائے گا جس کے بعد یہاں سے ملبہ بھی ہٹایا جائے گا تو وہ آپریشن جو ہے اب شروع کر دیا گیا ہے لیکن کسی افراد کے اندر ہونے کی اب تک جو ہے وہ اطلاعات نہیں کیونکہ انکا کہنا ہے کہ جو  ریلیٹیوز اور جو آس پاس کے لوگ تھے ان سے تمام لسٹ مانگی گئی اور اسکے مطابق جو یہ آپریشن ریسکیو کا میں آپ سے یہی سوال ک

...

🎧 Transcribing…                                           
Loading audio.wav ...
Audio duration: 81.8s

===== RESULT =====
GOLD: ہماری بی آر ٹی کا سب سے بڑا پروجیکٹ ہے وہ بی آر ٹی ریڈ لائن ہے تھوڑا سا اس کے بارے میں لوگ سننا چاہتے ہیں ابھی کیونکہ نیپا سے وہ راستہ بھی بند ہے اس وجہ سے اور مشکلات پیدا ہوئ ہیں تو بی آر ٹی ریڈ لائن کیا پراجیکٹ ہے بی آر ٹی ریڈ لائن کیا ہے آپ نے جمالی صاحب بات کی میں ابھی بھی آرہا ہوں تھوڑا لیٹ ہو گیا ایشین ڈیولپمنٹ بینک کے ایک وفد سے میری ملاقات تھی اس میں جو حائل جو پریشانیاں ہیں وہ ہمارے میں وہ کھل کر بتا دوں ہمارے کانٹریکٹرز کے ساتھ ہمارے تنازعات تھے اس کی سیٹلمنٹ بھی کر رہے ہیں آل موست ہم لوگ اس سیٹلمنٹ کے بلکل نائنٹی نائن پرسنٹ قریب آ چکے ہیں اور اس کی تفصیلی بریفنگ ہے کسی دن میں ایک اور بھی آپ کے توسط سے بتانا چاہوں گا کہ میں چند دنوں میں ایک بریفنگ رکھتا ہوں جس میں میڈیا کے لوگوں کو بلاتا ہوں پولیٹیکل پارٹیز کو بلاتا ہوں ایم کیو ایم ہے جماعت اسلامی ہے پی ٹی آئی ہے جو پولیٹیکل پارٹیز جے یو آئی ایف جو کراچی میں اپنا اسٹیک سمجھتی ہیں ان سب کو بلاتا ہوں 

🎧 Transcribing…                                           
Loading audio.wav ...
Audio duration: 73.5s

===== RESULT =====
GOLD: سندھ ہائی کورٹ نے کے ایم سی کو کینٹونمنٹ کے عالقوں کے رہائشیوں سے بجلی کے بلوں میں میونسپل ٹیکس وصول کرنے سے
روک دیا ہے عدالت نے کینٹونمنٹ عالقوں سے میونسپل ٹیکس کی وصولی کینٹونمنٹ بورڈز کا اختیار قرار دیا ہے تفصیل جانیں گے
جیو نیوز کے نمائندے امین انور سے امین بتائیے گا کارروائی کے حوالے سے کینٹونمنٹ بورڈ ملیر کے رہائشیوں کی درخواست پر
سماعت کی ہےاورعدالت نےکے ایم سی کو کینٹونمنٹ عالقوں سے ایم یوسی پی چارجزوصول کرنے سے روک دیا ہےعدالت نے کہا ہے
کہ کینٹونمنٹ عالقوں سے میونسپل ٹیکس کی وصولی کینٹونمنٹ بورڈز کا اختیار ہے کے ایم سی کو بجلی کے بلوں میں ایم یوسی
پی چارجز وصول کرنے کا کوئی اختیار نہیں ہے درخواست گزاروں کے وکیل عثمان فاروق کا مؤقف تھا کہ ملیرکینٹونمنٹ بورڈ
پہلے ہی کنزرویسی ٹیکس وصول کر رہا ہے اور کے ایم سی کی جانب سے بجلی کے بلوں میں میونسپل چارجز بھی شامل کر دیے
گئے ہیں بیک وقت دو ادارے میونسپل ٹیکس وصول کر رہے ہیں جو کہ قانون کی بھی خالف ورزی ہےاور

🎧 Transcribing…                                           
Loading audio.wav ...
Audio duration: 83.1s

===== RESULT =====
GOLD: ﻿ بریکنگ نیوز ہے کراچی کے علاقے ناظم آباد میں گاڑی کی ٹکر سے ایک شخص جاں بحق ہو گیا ہے اس پر مزید کیا تفصیلات ہیں جانیں گے کراچی میں جیو نیوز کے نمائندے قمر علی مستوئی سے قمر بتائے گا حادثہ کیسے پیش آیا ہے اور جاں بحق شخص کی شناخت ہو سکی ہے کیا؟ جی اس حوالے سے ریسکیو کا کہنا ہے کہ نامعلوم گاڑی کی ٹکر سےجو ہے وہ یہ شخص جاں بحق ہوا ہے جس کی فوری طور پر شناخت نہیں ہو سکی ہے جاں بحق ہو نے والے شخص کی لاش کو عباسی شہید ہسپتال منتقل کیا گیا ہے اس کے کپڑوں سے کوئی شناختی دستاویزات جو ہے وہ نہیں ملی پولیس نے جو ہے وہ متوفی کو ٹکر مارنے والی گاڑی کی تلاش شروع کر دی ہے اور جائے حادثہ  سےجو ہےوہ پولیس کی اطراف میں موجود شہریوں سےجو ہے وہ مرنے والے شخص کے متعلق معلومات حاصل کرنے کی کوشش کی کر رہی ہے تاکہ اس کی شناخت ہو سکے دوسری جانب بھینس کالونی نمبر دس پر بھی جو ہے وہ ٹریفک حادثہ پیش آیا ہے پولیس کے مطابق جو ہے وہ تیز رفتار ٹرالر ڈرائیور کے قابو سے باہر ہو کر الٹ گیا ج

🎧 Transcribing…                                           
Loading audio.wav ...
Audio duration: 70.7s

===== RESULT =====
GOLD: اہم خبر آپ کو دیں گے کراچی میں سپر ہائی وے ایم نائن پر ڈمپر اور وین سمیت متعدد گاڑیوں کی ٹکر ہوئی ہے حادثے میں خاتون سمیت چار افراد جاں بحق ہو گئے ہیں دانیال سید نمائیدہ جیو نیوز ہمارے ساتھ موجود ہیں دانیال کیا واقعہ پیش آیا ہے جی دیکھیں بالکل موٹر وے ایم نائن پر کراچی آتے ہوئے یہ خوفناک ٹریفک حادثہ ہوا ہے بارش کے باعث پسلن تھی اور ڈھلان سے گاڑیاں اتر رہی تھیں تو اس دوران تیز رفتاری کے باعث ڈمپر وین اور متعدد گاڑیاں آپس میں ٹکرا گئیں موقع پر پولیس اور ریسکیو ادارے پہنچ گئے ریسکیو حکام کے مطابق چار افراد موقع پر جاں بحق ہو گئے ہیں جبکہ متعدد افراد زخمی ہوئے ہیں آئی جی سندھ نے بھی واقعے کا نوٹس لیا ہے ایس ایس پی ملیر کو فوری طور پر موقع پر پہنچنے کی ہدایت کی گئی ہے موٹر وے پولیس موقع پر موجود ہے تمام زخمی  جو ہیں انہیں قریبی اسپتال منتقل کر دیا گیا ہے خاتون اور بچے سمیت چار افراد جاں بحق ہوئے ہیں جبکہ خواتین سمیت چھ افراد زخمی بتائے جا رہے ہیں جاں بحق افراد میں

🎧 Transcribing…                                           
Loading audio.wav ...
Audio duration: 99.1s

===== RESULT =====
GOLD: رضا رومی صاحب ایک طرف صحت لے کر احتجاج ہے ماضی کی تلخ یادیں بھی ہیں آج مریم بی بی نے پرانی ویڈیوز چلائیں یوں کہیے کہ عمران خان آج پورے پاکستان کے ٹی وی چینلز پر دکھائی دیے اور ان کی وہ گفتگو سنوائی گئی اس کا مطلب ہے کہ تلخیاں ابھی اندر ویسے ہی موجود ہیں کیا اس دوران اختیار کیے گئے رویے جب نواز شریف یا مرحومہ کلثوم نواز صاحبہ کی بیماری کے معاملات چل رہے تھے وہ آج کے فیصلوں پر سایہ ڈال رہے ہیں عائشہ  بالکل ڈال رہے ہیں دیکھیں یہی تو بدقسمتی سے دائرے کا سفر ہے آپ پاکستان کی تاریخ کا وہ آج بھی جاری ہے اور یہ جو جس واقعے کا آپ ذکر کر رہی ہیں جس مسئلے کو ہم زیر بحث لا رہے ہیں اس میں بھی یہی چیز جو ہے یہی چیز بار بار کہی جا رہی ہے موجودہ حکومت کے اندر سے آپ نے دیکھا کہ پریزیڈنٹ صاحب نے ابھی دو روز پہلے ایک بہت ہی سخت تقریر کی اور اس میں انہوں نے عمران خان کی جیل کی جو ہے صورتحال اس کے اوپر بھی کومنٹ کیا اور اس کومنٹ میں ایک بات پر مجھے بہت اعتراض ہے کہ مرد اپنی کمزور

In [14]:
from pathlib import Path
import pandas as pd

def safe_dirname(s: str) -> str:
    return (
        s.strip()
         .lower()
         .replace(" ", "_")
         .replace("/", "_")
    )

def write_outputs_and_update_csv(manifest_df: pd.DataFrame,
                                 names: list,
                                 preds: list,
                                 wers_list: list,
                                 model_column: str,
                                 transcripts_root: Path,
                                 gold_file_col: str,
                                 out_csv_path: Path):
    # ---- 1) ensure column exists ----
    if model_column not in manifest_df.columns:
        raise ValueError(f"MODEL_COLUMN '{model_column}' not found in CSV. Columns: {list(manifest_df.columns)}")

    # ---- 2) write WER into the chosen column ----
    # store as percent like your sheet (e.g., 23.32%)
    manifest_df.iloc[:len(wers_list), manifest_df.columns.get_loc(model_column)] = [f"{w*100:.2f}%" for w in wers_list]

    # ---- 3) save transcriptions into a model folder ----
    model_dir = transcripts_root / safe_dirname(model_column)
    model_dir.mkdir(parents=True, exist_ok=True)

    for i, (name, pred) in enumerate(zip(names, preds)):
        gold_filename = str(manifest_df.iloc[i][gold_file_col]).strip()

        # use SAME FILE NAME but force .txt output
        base = Path(gold_filename).stem      # "CTD_Gold" from "CTD_Gold.txt"
        out_txt = model_dir / f"{base}.txt"

        out_txt.write_text(pred.strip(), encoding="utf-8")

    # ---- 4) save updated CSV ----
    out_csv_path.parent.mkdir(parents=True, exist_ok=True)
    manifest_df.to_csv(out_csv_path, index=False, encoding="utf-8")
    print(f"✅ Updated CSV saved to: {out_csv_path.resolve()}")
    print(f"✅ Transcriptions saved in: {model_dir.resolve()}")

# ---- RUN POST-PROCESS ----
OUT_CSV = Path("fyp-summer/youtube_videos_dataset") / "youtube_videos_with_wers.csv"

write_outputs_and_update_csv(
    manifest_df=manifest_df,
    names=names,
    preds=preds,
    wers_list=wers_list,
    model_column=MODEL_COLUMN,
    transcripts_root=TRANSCRIPTS_ROOT,
    gold_file_col=GOLD_FILE_COL,
    out_csv_path=OUT_CSV
)

✅ Updated CSV saved to: /home/shaider/whisper-UrduASR/fyp-summer/youtube_videos_dataset/youtube_videos_with_wers.csv
✅ Transcriptions saved in: /home/shaider/whisper-UrduASR/fyp-summer/youtube_videos_dataset/whisper_turbo_ft


/data/tmp/ipykernel_2537731/2903400270.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['30.32%', '22.92%', '23.37%', '23.04%', '25.85%', '25.99%', '34.74%', '18.34%', '17.65%', '35.12%', '9.35%', '21.61%', '21.94%', '41.27%', '12.65%', '33.69%', '17.24%', '25.71%', '33.78%', '72.90%', '10.82%', '30.41%', '48.85%', '15.93%', '25.66%', '23.34%', '35.94%', '27.70%', '30.04%', '11.46%']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  manifest_df.iloc[:len(wers_list), manifest_df.columns.get_loc(model_column)] = [f"{w*100:.2f}%" for w in wers_list]
